<a href="https://colab.research.google.com/github/nbaulib/queens/blob/main/F22_HC7_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HC7 & HC8 Data Exploration

Below you will find a first exploration of the `Affordable_Housing_Production_by_Building.csv` dataset. The goal is for you to learn the tools for data exploration (many of these you learned in our online labs) and then be able to further explore the data and obtain results to report in our final presentation.  

As you know, we must first import the packages we will use:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## EXPLORE THE DATA

To begin, we will read the data into a pandas dataframe and find out its shape (dimensions)

In [ ]:
# download the file from our drive
!gdown https://drive.google.com/uc?id=1d1P7EhTQyYd6woSsQ-EPpu0BNJCXyDvn
#list the files in the current directory to confirm the file is there
!ls


In [ ]:
# read Affordable_Housing_Production_by_Building.csv into a dataframe
affordable = pd.read_csv("Affordable_Housing_Production_by_Building.csv")

print("The dimension of the table is: ", affordable.shape)

Alternatively, to find out how many observations (rows) in the dataset:

In [ ]:
print("Number of rows: ", len(affordable))

To find out the names of the columns, we can print the header of the dataframe (with zero rows)

In [ ]:
print("Column Names:\n",affordable.head(0))

Alternatively, to see sample data, we can print the header of the dataframe with the first two rows:

In [ ]:
print("Column Names:\n",affordable.head(2))

Oh no! There are too many columns to print, 41 of them! (thats why there is the `...` in between.)  
Let's look at what type of data is stored in each column:

In [ ]:
affordable.info()

You can access speicfic columns using exact the same name included in quotes and get the head of the dataframe with the selected columns (as shown below)

In [ ]:
print("Project Name:\n",affordable['Project Name'].head(5))

We can use `describe()` to get some stats for the numerical data, shown as follows. Some of the output make sense, for example, calculate the average, maximum, or minimum number of 2-BR Units, but some of the output will not be used, for example, the mean of Postcode.

In [ ]:
affordable.describe()

The function `describe()` gives the number of values (`count`), average, standard deviation, minimum and maximum values as well as the 25th, 50th and 75th percentiles, **for each numerical column**.
Alternatively, to retrieve and print the stats **for a particular column**:

In [ ]:
num_stats = affordable.describe()
print("Statistics for '2-BR Units' column:\n",num_stats['2-BR Units'])

The `describe()` function, as called above, provides statistics for numerical data. To get some information for the categorical data:

In [ ]:
affordable.describe( include=['O'])

The `describe()` function with `include=['O']` argument (that is a capital letter O), will provide the number of values (`count`), the number of unique values, the most frequent value (`top`) and the frequency of that value, **for each categorical column**.

As we did above, if you want to extract the stats **for a particular categorical column**:

In [ ]:
cat_stats = affordable.describe(include=['O'])
print("Borough:\n", cat_stats['Borough'])

Here we can see that all rows specify the `Borough` (count is the same as the number of rows in the dataset, which is 5798 as we saw earlier). We can also observe that there are 5 boroughs (unique == 5) and that Brooklyn (marked by top) has most projects (2364).

Let's look at another column:

In [ ]:
cat_stats = affordable.describe(include=['O'])
print("Statistics for Building Completion Date:\n", cat_stats['Building Completion Date'])

Here we also see that not all rows have an entry for `Building Completion Date` (count is less than number of rows in the dataset), we see that there are 1327 (marked as unique) different dates, the date with the most projects completed (`top`) is 06/20/2019, with 103 buildings completed on that day.

Now we may wonder, what is the latest date for `Building Completion Date` in our dataset? To get the answer, first we convert the date (which is now a string) to a datetime format recognized by pandas, and then we can ask pandas for the latest (`max`) date:

In [ ]:
affordable["Building Completion Date"] = pd.to_datetime(affordable["Building Completion Date"].apply(str))
print("Latest date for Building Completion Date: ", affordable['Building Completion Date'].max())

## CLEAN THE DATA  

It is always a good idea to clean your data before you start working. The most common problem you will encouter is missing data. If some rows do not provide a value for a particular feature (column), you may encounter errors when you try to compute statistics or plot the data.   

To check if there are any missing values by suming up the null values for each column:

In [ ]:
print("Number of dataponts with null entry for each column:\n",affordable.isnull().sum())

Here we see that there are many columns that have issing values (non-zero).

To find out if there are same project names, we can use `value_counts()` on that column

In [ ]:
affordable['Project Name'].value_counts()

Indeed, there are multiple records with the same project names, for example, there are 79 occurrences of "JOE CENTRAL BROOKLYN LLC.YR15.FY19" (without quotes) in column Project Name.

So now, **let's clean the data** by removing the null values.  
We can do that by replacing null values for our numerical columns with 0

In [ ]:
affordable.fillna({'Building ID':0, 'Postcode':0,'BBL':0, 'BIN':0},inplace=True)
affordable.fillna({'Council District':0},inplace=True)
affordable.fillna({'Latitude':0, 'Longitude':0},inplace=True)
affordable.fillna({'Latitude (Internal)':0, 'Longitude (Internal)':0},inplace=True)

And replace null values in our categorical data (string values) with the empty string:

In [ ]:
affordable.fillna({'Census Tract':"",'NTA - Neighborhood Tabulation Area':""},inplace=True)

Now we can check to make sure we got rid of all null values:

In [ ]:
print(affordable.isnull().sum())

And voila', our data that are of **number** or **string** type are free of null values!!! Null values of **date/time** are not changed.  

Another sensible thing to do, and it should always be considered, is whether our data should be anonymized for privacy protection. Also, when working with data, it is always important to think whether we really need for our analysis the data that could potentially identify idividual subjects, and if not, it is always a good idea to remove that data all toghether.  

In this case the data comes from NYC Open data and it has already been anonymized somewhat, but if you think some data involves privacy or you do not need it, it is a good idea to remove those columns. In our examples, we would like to remove Latitude (Internal) and Longitude (Internal).

In [ ]:
affordable.drop(['Latitude (Internal)'], axis=1, inplace=True)
affordable.drop(['Longitude (Internal)'], axis=1, inplace=True)

In [ ]:
print("Column Names:\n",affordable.head(0))

And we can see that we no longer have columns that we won't be using!

## VISUALIZE THE DATA

### Let's look at the data by borough:  

To find out the names of the boroughs (we kind of know this already...):

In [ ]:
print("Unique Borough Names:\n", affordable.Borough.unique())

Let's group the data by borough (`Borough`)

In [ ]:
boro_group = affordable.groupby(['Borough'])

Now that the data is grouped by borough in boro_group, let's plot the data to observe the average 1-BR units per borough:

In [ ]:
boro_group['1-BR Units'].mean().plot.bar()

To save your plot in an image file you can do the following to leave more space at the bottom of the figure to fully visualize the names of the boroughs on the x axis, label the y axis, and then clear the figure so we can make more plots later:

In [ ]:
#create more space for x-axis labels
plt.gcf().subplots_adjust(bottom=0.25)
plt.ylabel('Average 1-BR Units')
#get the current figure
fig = plt.gcf()
#save figure to an image file
fig.savefig('mean1BRUnits.png')

#clear figure to generate next pyplot
plt.clf()


Let's now plot the data to observe Reporting Construction Type by borough:

In [ ]:
#Generate the number of Preservation or New Construction with value_count()
boro_group['Reporting Construction Type'].value_counts().plot.bar()

Let's save this plot to a file (we already adjusted the bottom of the fiugre, so we don't need to repeat that):

In [ ]:
#label the y-axis
plt.ylabel('Number of Buildings')
#get the current figure
fig2 = plt.gcf()
#save figure to an image file
fig2.savefig('constructionTypeByBorough.png')

#clear figure to generate next pyplot
plt.clf()

Brooklyn has the most new construction.



Let's look at the 1-BR Units Percentage by borough:

In [ ]:
boro_group['Prevailing Wage Status'].value_counts().plot.bar()

Let's save this plot to a file:

In [ ]:
#label the y-axis
plt.ylabel('Number of Buildngs')
#get the current figure
fig2 = plt.gcf()
#save figure to an image file
fig2.savefig('boroughPrevailingWageStatus.png')

#clear figure to generate next pyplot
plt.clf()

We can observe all the data from each borough but some of them might be hard to see due to the large differences in the number of buildings in Brooklyn.

Let's say I now want to look at data from Staten Island only (**note I am selecting rows here!!!**)

In [ ]:
#Select only buildings in Staten Island
st = affordable[affordable['Borough'].isin(['Staten Island'])]
print("Number of buildings in Staten Island: ", len(st))

I now have a new dataframe called st with only rows where borough == 'Staten Island", and I can see that there are 255 buildings for Staten Island.

In [ ]:
#Graph data from only Staten Island
st['Prevailing Wage Status'].value_counts().plot.bar()

Let's save this plot to a file:

In [ ]:
#label the y-axis
plt.ylabel('Num of buildings in Staten Island')
#get the current figure
fig2 = plt.gcf()
#save figure to an image file
fig2.savefig('statenIslandPrevailingWageStatus.png')
#clear figure to generate next pyplot
plt.clf()



### YOUR TURN!

Now that you know how to explore the data, clean the data, obtain statistics about the data, visualize the data and select a subset of the data based on the value in a particular column (e.g. neighbourhood_group == 'Staten Island"), think about how you want to explore the data for your analysis:    
  1. As a group, think about an overall data-driven discussion of your borough and how it compares to the others.  
  2. Individually, analyze the data in your neighborhood and compare to the data for the borough.
  
As you explore your data, keep in mind your observations and conclusions from HC2 and HC3 and see if you can make any connections, or if you find that the data supports those conclusions.

---
---
---

Feel the need to dig in more? Check this out!

In [ ]:
park = affordable[affordable['Project Name'].str.lower().str.contains('park')]

In [ ]:
park.head(5)